# SVMs classifier 

Links to papers and related information:

file:///C:/Users/ainav/OneDrive/Escritorio/Saudi_License_Plate_Recognition_Algorith.pdf
file:///C:/Users/ainav/OneDrive/Escritorio/Design_of_Automatic_Number_Plate_Recogni.pdf
https://www.academia.edu/110187161/SVM_Based_License_Plate_Recognition_System?nav_from=36dcd588-c2c0-48f5-bf6c-6a6e18d1b73d
